# Amazon Neptune with Cypher

>[Amazon Neptune](https://aws.amazon.com/neptune/) 是一款高性能图分析和无服务器数据库，具有卓越的可伸缩性和可用性。
>
>本示例展示了使用 `openCypher` 查询 `Neptune` 图数据库并返回人类可读响应的 QA 链。
>
>[Cypher](https://en.wikipedia.org/wiki/Cypher_(query_language)) 是一种声明式图查询语言，允许在属性图中进行富有表现力和高效的数据查询。
>
>[openCypher](https://opencypher.org/) 是 Cypher 的开源实现。# Neptune Open Cypher QA Chain
此 QA 链使用 openCypher 查询 Amazon Neptune 并返回人类可读的响应。

LangChain 支持使用 `create_neptune_opencypher_qa_chain` 来支持 [Neptune Database](https://docs.aws.amazon.com/neptune/latest/userguide/intro.html) 和 [Neptune Analytics](https://docs.aws.amazon.com/neptune-analytics/latest/userguide/what-is-neptune-analytics.html)。

Neptune Database 是一款无服务器图数据库，旨在实现最佳的可伸缩性和可用性。它为需要扩展到每秒 100,000 次查询、多可用区高可用性和多区域部署的图数据库工作负载提供解决方案。您可以使用 Neptune Database 来构建社交网络、欺诈预警和客户画像 360 应用程序。

Neptune Analytics 是一款分析数据库引擎，可以快速分析内存中的大量图数据，以获得洞察和发现趋势。Neptune Analytics 为快速分析现有图数据库或存储在数据湖中的图数据集提供解决方案。它使用流行的图分析算法和低延迟分析查询。

## 使用 Neptune Database

In [ ]:
from langchain_aws.graphs import NeptuneGraph

host = "<neptune-host>"
port = 8182
use_https = True

graph = NeptuneGraph(host=host, port=port, use_https=use_https)

### 使用 Neptune Analytics

In [ ]:
from langchain_aws.graphs import NeptuneAnalyticsGraph

graph = NeptuneAnalyticsGraph(graph_identifier="<neptune-analytics-graph-id>")

## 使用 Neptune openCypher QA 链

该 QA 链使用 openCypher 查询 Neptune 图数据库，并返回人类可读的响应。

In [12]:
from langchain_aws import ChatBedrockConverse
from langchain_aws.chains import create_neptune_opencypher_qa_chain

MODEL_ID = "anthropic.claude-3-5-sonnet-20241022-v2:0"
llm = ChatBedrockConverse(
    model=MODEL_ID,
    temperature=0,
)

chain = create_neptune_opencypher_qa_chain(llm=llm, graph=graph)

result = chain.invoke("How many outgoing routes does the Austin airport have?")
print(result["result"].content)

Austin airport has 98 outgoing routes.


### 添加消息历史记录

Neptune openCypher QA 链可以通过 [`RunnableWithMessageHistory`](https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html#langchain_core.runnables.history.RunnableWithMessageHistory) 进行包装。这会为链添加消息历史记录，使我们能够创建一个在多次调用中保留对话状态的聊天机器人。

首先，我们需要一种存储和加载消息历史记录的方法。为此，每个线程都将创建为 [`InMemoryChatMessageHistory`](https://python.langchain.com/api_reference/core/chat_history/langchain_core.chat_history.InMemoryChatMessageHistory.html) 的实例，并存储在字典中以便重复访问。

(另请参阅：https://python.langchain.com/docs/versions/migrating_memory/chat_history/#chatmessagehistory)

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory

chats_by_session_id = {}


def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    chat_history = chats_by_session_id.get(session_id)
    if chat_history is None:
        chat_history = InMemoryChatMessageHistory()
        chats_by_session_id[session_id] = chat_history
    return chat_history

现在，QA 链（QA chain）和消息历史存储（message history storage）可以用来创建新的 `RunnableWithMessageHistory`。请注意，我们必须将 `query` 设置为输入键，以匹配基础链（base chain）的预期格式。

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory

runnable_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="query",
)

在调用链之前，需要为对话生成一个唯一的 `session_id`，新的 `InMemoryChatMessageHistory` 将会记住这个 ID。

In [ ]:
import uuid

session_id = uuid.uuid4()

最后，使用 `session_id` 调用启用消息历史记录的链。

In [8]:
result = runnable_with_history.invoke(
    {"query": "How many destinations can I fly to directly from Austin airport?"},
    config={"configurable": {"session_id": session_id}},
)
print(result["result"].content)

You can fly directly to 98 destinations from Austin airport.


随着链使用相同的 `session_id` 被调用，响应将结合之前对话查询的上下文返回。

In [9]:
result = runnable_with_history.invoke(
    {"query": "Out of those destinations, how many are in Europe?"},
    config={"configurable": {"session_id": session_id}},
)
print(result["result"].content)

You can fly directly to 4 destinations in Europe from Austin airport.


In [10]:
result = runnable_with_history.invoke(
    {"query": "Give me the codes and names of those airports."},
    config={"configurable": {"session_id": session_id}},
)
print(result["result"].content)

The four European destinations you can fly to directly from Austin airport are:
- AMS (Amsterdam Airport Schiphol)
- FRA (Frankfurt am Main)
- LGW (London Gatwick)
- LHR (London Heathrow)
